# Setup

In [1]:
packrat::init(project = "/work/postresearch/Shared/Projects/Farbod/CaseMix")

Initializing packrat project in directory:
- "/work/postresearch/Shared/Projects/Farbod/CaseMix"

Initialization complete!



In [2]:
#packrat::snapshot()

In [3]:
packrat::status()

Up to date.



In [4]:
setwd('/work/postresearch/Shared/Projects/Farbod/Integration_Intensity')

In [5]:
options(repr.matrix.max.rows=100, repr.matrix.max.cols=300)
options(repr.plot.width = 20, repr.plot.height = 15)
options(width=300)
    
numcores=56

library(tidyverse)
library(parallel)
library(data.table)
library(fst)
library(comorbidity)
library(zeallot)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
library(icd)
`%!in%` = Negate(`%in%`)

setDTthreads(numcores)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


This is {comorbidity} version 1.0.3.

A lot has changed since the pre-1.0.0 release on CRAN, please check-out breaking changes here:

-> https://ellessenne.github.io/comorbidity/articles/C-changes.html


Attaching package: ‘reshape’


The following object is masked fro

# Finding the diagnosis for each patient

## Carrier files

In [108]:
carrier_data_2013=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2013.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2014=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2014.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2015=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2015.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2016=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2016.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2017=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2017.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2018=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2018.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2019=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2019.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))
carrier_data_2020=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2020.fst",as.data.table = T, columns = c("DESY_SORT_KEY","LINE_ICD_DGNS_CD"))

## Inpatient files

In [111]:
inpatient_data_2013 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2013.fst",as.data.table = T)
inpatient_data_2014 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2014.fst",as.data.table = T)
inpatient_data_2015 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2015.fst",as.data.table = T)
inpatient_data_2016 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2016.fst",as.data.table = T)
inpatient_data_2017 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2017.fst",as.data.table = T)
inpatient_data_2018 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2018.fst",as.data.table = T)
inpatient_data_2019 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2019.fst",as.data.table = T)
inpatient_data_2020 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2020.fst",as.data.table = T)

## Outpatient files

In [112]:
outpatient_data_2013 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2013.fst",as.data.table = T)
outpatient_data_2014 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2014.fst",as.data.table = T)
outpatient_data_2015 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2015.fst",as.data.table = T)
outpatient_data_2016 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2016.fst",as.data.table = T)
outpatient_data_2017 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2017.fst",as.data.table = T)
outpatient_data_2018 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2018.fst",as.data.table = T)
outpatient_data_2019 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2019.fst",as.data.table = T)
outpatient_data_2020 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2020.fst",as.data.table = T)

In [109]:
carrier_test=
carrier_data_2020[1:10,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

In [113]:
head(carrier_test)

MBI,DIAG
<int>,<chr>
100000015,H903
100000015,H3530
100000015,H3530
100000015,H353231
100000015,H353231
100000015,E119


In [114]:
find_all_diagnosis=function(data){
    data=data[,.(
    DESY_SORT_KEY,
    PRNCPAL_DGNS_CD,
    ICD_DGNS_CD1,
    ICD_DGNS_CD2,
    ICD_DGNS_CD3,
    ICD_DGNS_CD4,
    ICD_DGNS_CD5,
    ICD_DGNS_CD6,
    ICD_DGNS_CD7,
    ICD_DGNS_CD8,
    ICD_DGNS_CD9,
    ICD_DGNS_CD10,
    ICD_DGNS_CD11,
    ICD_DGNS_CD12,
    ICD_DGNS_CD13,
    ICD_DGNS_CD14,
    ICD_DGNS_CD15,
    ICD_DGNS_CD16,
    ICD_DGNS_CD17,
    ICD_DGNS_CD18,
    ICD_DGNS_CD19,
    ICD_DGNS_CD20,
    ICD_DGNS_CD21,
    ICD_DGNS_CD22,
    ICD_DGNS_CD23,
    ICD_DGNS_CD24,
    ICD_DGNS_CD25
    )]
    
    result=melt(data,id.vars = "DESY_SORT_KEY")%>%as.data.table()
    
    result=result[,.(MBI=DESY_SORT_KEY,
                     DIAG=value
                    )]
    
    result=result[nchar(DIAG)>0,]
    
    return(result)
    
    
}

In [115]:
in_test=find_all_diagnosis(inpatient_data_2020[1:2,])

In [116]:
out_test=find_all_diagnosis(outpatient_data_2020[1:2,])

In [117]:
combine_diagnosis=function(carrier,outpatient,inpatient){
    data=rbind(carrier,outpatient,inpatient)
    data=data[order(MBI)]
    return(data)
}

In [118]:
diag_test=combine_diagnosis(carrier_test,out_test,in_test)

In [119]:
head(diag_test)

MBI,DIAG
<int>,<chr>
100000015,H903
100000015,H3530
100000015,H3530
100000015,H353231
100000015,H353231
100000015,E119


In [123]:
carrier_2013=
carrier_data_2013[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2014=
carrier_data_2014[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2015=
carrier_data_2015[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2016=
carrier_data_2016[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2017=
carrier_data_2017[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2018=
carrier_data_2018[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2019=
carrier_data_2019[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

carrier_2020=
carrier_data_2020[,.(
    MBI=DESY_SORT_KEY,
    DIAG=LINE_ICD_DGNS_CD)]

In [124]:
head(carrier_2013)

MBI,DIAG
<int>,<chr>
100000015,496
100000015,496
100000015,496
100000015,41400
100000015,41400
100000015,496


In [125]:
in_2013=find_all_diagnosis(inpatient_data_2013)
in_2014=find_all_diagnosis(inpatient_data_2014)
in_2015=find_all_diagnosis(inpatient_data_2015)
in_2016=find_all_diagnosis(inpatient_data_2016)
in_2017=find_all_diagnosis(inpatient_data_2017)
in_2018=find_all_diagnosis(inpatient_data_2018)
in_2019=find_all_diagnosis(inpatient_data_2019)
in_2020=find_all_diagnosis(inpatient_data_2020)

In [126]:
head(in_2013)

MBI,DIAG
<int>,<chr>
100000099,71536
100000099,99812
100000315,0090
100000559,7907
100000905,5990
100001177,1889


In [127]:
out_2013=find_all_diagnosis(outpatient_data_2013)
out_2014=find_all_diagnosis(outpatient_data_2014)
out_2015=find_all_diagnosis(outpatient_data_2015)
out_2016=find_all_diagnosis(outpatient_data_2016)
out_2017=find_all_diagnosis(outpatient_data_2017)
out_2018=find_all_diagnosis(outpatient_data_2018)
out_2019=find_all_diagnosis(outpatient_data_2019)
out_2020=find_all_diagnosis(outpatient_data_2020)

In [128]:
head(out_2013)

MBI,DIAG
<int>,<chr>
100000015,496
100000015,496
100000015,V0481
100000019,53510
100000019,1541
100000019,1540


In [129]:
diag_2013=combine_diagnosis(carrier_2013,out_2013,in_2013)
diag_2014=combine_diagnosis(carrier_2014,out_2014,in_2014)
diag_2015=combine_diagnosis(carrier_2015,out_2015,in_2015)
diag_2016=combine_diagnosis(carrier_2016,out_2016,in_2016)
diag_2017=combine_diagnosis(carrier_2017,out_2017,in_2017)
diag_2018=combine_diagnosis(carrier_2018,out_2018,in_2018)
diag_2019=combine_diagnosis(carrier_2019,out_2019,in_2019)
diag_2020=combine_diagnosis(carrier_2020,out_2020,in_2020)

In [132]:
head(diag_2013)

MBI,DIAG
<int>,<chr>
100000015,496
100000015,496
100000015,496
100000015,41400
100000015,41400
100000015,496


In [133]:
write.csv(diag_2013,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2013.csv")
write.csv(diag_2014,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2014.csv")
write.csv(diag_2015,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2015.csv")
write.csv(diag_2016,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2016.csv")
write.csv(diag_2017,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2017.csv")
write.csv(diag_2018,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2018.csv")
write.csv(diag_2019,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2019.csv")
write.csv(diag_2020,"/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2020.csv")

# Converting to HCC codes

In [8]:
diag_2013=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2013.csv", num_threads = numcores)%>% as.data.table()
diag_2014=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2014.csv", num_threads = numcores)%>% as.data.table()
diag_2015=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2015.csv", num_threads = numcores)%>% as.data.table()
diag_2016=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2016.csv", num_threads = numcores)%>% as.data.table()
diag_2017=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2017.csv", num_threads = numcores)%>% as.data.table()
diag_2018=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2018.csv", num_threads = numcores)%>% as.data.table()
diag_2019=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2019.csv", num_threads = numcores)%>% as.data.table()
diag_2020=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/diag_2020.csv", num_threads = numcores)%>% as.data.table()

New names:
• `` -> `...1`
Rows: 134286683 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): DIAG
dbl (2): ...1, MBI

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 136922602 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): DIAG
dbl (2): ...1, MBI

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 139859054 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): DIAG
dbl (2): ...1, MBI

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet thi

In [32]:
diag_2013[,date:=as.IDate(paste("2013","-01","-01",sep=""),"%Y-%m-%d")]
diag_2014[,date:=as.IDate(paste("2014","-01","-01",sep=""),"%Y-%m-%d")]
diag_2015[,date:=as.IDate(paste("2015","-01","-01",sep=""),"%Y-%m-%d")]
diag_2016[,date:=as.IDate(paste("2016","-01","-01",sep=""),"%Y-%m-%d")]
diag_2017[,date:=as.IDate(paste("2017","-01","-01",sep=""),"%Y-%m-%d")]
diag_2018[,date:=as.IDate(paste("2018","-01","-01",sep=""),"%Y-%m-%d")]
diag_2019[,date:=as.IDate(paste("2019","-01","-01",sep=""),"%Y-%m-%d")]
diag_2020[,date:=as.IDate(paste("2019","-01","-01",sep=""),"%Y-%m-%d")]

In [17]:
diag_2013[,MBI:=as.character(MBI)]
diag_2014[,MBI:=as.character(MBI)]
diag_2015[,MBI:=as.character(MBI)]
diag_2016[,MBI:=as.character(MBI)]
diag_2017[,MBI:=as.character(MBI)]
diag_2018[,MBI:=as.character(MBI)]
diag_2019[,MBI:=as.character(MBI)]
diag_2020[,MBI:=as.character(MBI)]

In [27]:
head(diag_2013)
tail(diag_2020)

...1,MBI,DIAG,date
<dbl>,<chr>,<chr>,<date>
1,100000015,496,2013-01-01
2,100000015,496,2013-01-01
3,100000015,496,2013-01-01
4,100000015,41400,2013-01-01
5,100000015,41400,2013-01-01
6,100000015,496,2013-01-01


...1,MBI,DIAG,date
<dbl>,<chr>,<chr>,<date>
123643420,499999981,W57XXXA,2020-01-01
123643421,499999981,W57XXXD,2020-01-01
123643422,499999981,R634,2020-01-01
123643423,499999981,R634,2020-01-01
123643424,499999981,Z6824,2020-01-01
123643425,499999981,Z6824,2020-01-01


In [34]:
comorbid_hcc(as.data.frame(diag_2013[MBI==100000015,]),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )

,MBI,date,hcc
,<chr>,<date>,<dbl>
1,100000015,2013-01-01,111
2,100000015,2013-01-01,124


In [35]:
comorbid_hcc(as.data.frame(diag_2015[MBI==100000015,]),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )

,MBI,date,hcc
,<chr>,<date>,<dbl>
1,100000015,2015-01-01,19
2,100000015,2015-01-01,111
3,100000015,2015-01-01,124


In [ ]:
hcc_diag_2013=comorbid_hcc(as.data.frame(diag_2013),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2014=comorbid_hcc(as.data.frame(diag_2014),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2015=comorbid_hcc(as.data.frame(diag_2015),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2016=comorbid_hcc(as.data.frame(diag_2016),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2017=comorbid_hcc(as.data.frame(diag_2017),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2018=comorbid_hcc(as.data.frame(diag_2018),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2019=comorbid_hcc(as.data.frame(diag_2019),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )
hcc_diag_2020=comorbid_hcc(as.data.frame(diag_2020),
             date_name = "date",
             visit_name = "MBI",
             icd_name = "DIAG"
            )

In [112]:
hcc_diag_2013=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2013.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2014=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2014.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2015=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2015.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2016=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2016.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2017=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2017.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2018=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2018.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2019=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2019.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2020=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/hcc_diag_2020.csv",num_threads = numcores)%>%as.data.table()

New names:
• `` -> `...1`
Rows: 3443527 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (3): ...1, MBI, hcc
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 3104411 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (3): ...1, MBI, hcc
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 2710744 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (3): ...1, MBI, hcc
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALS

In [113]:
hcc_diag_2013=hcc_diag_2013[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2014=hcc_diag_2014[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2015=hcc_diag_2015[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2016=hcc_diag_2016[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2017=hcc_diag_2017[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2018=hcc_diag_2018[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2019=hcc_diag_2019[,.(MBI=as.factor(MBI),hcc)]
hcc_diag_2020=hcc_diag_2020[,.(MBI=as.factor(MBI),hcc)]

In [122]:
hcc_diag_2013[MBI==100000015]
hcc_diag_2015[MBI==100000015]
hcc_diag_2020[MBI==100000015]

MBI,hcc
<fct>,<dbl>
100000015,111
100000015,124


MBI,hcc
<fct>,<dbl>
100000015,19
100000015,111
100000015,124


MBI,hcc
<fct>,<dbl>
100000015,18
100000015,19
100000015,111
100000015,124


In [116]:
hcc_diag_2013_wide = dcast(hcc_diag_2013,MBI~hcc,length)
hcc_diag_2014_wide = dcast(hcc_diag_2014,MBI~hcc,length)
hcc_diag_2015_wide = dcast(hcc_diag_2015,MBI~hcc,length)
hcc_diag_2016_wide = dcast(hcc_diag_2016,MBI~hcc,length)
hcc_diag_2017_wide = dcast(hcc_diag_2017,MBI~hcc,length)
hcc_diag_2018_wide = dcast(hcc_diag_2018,MBI~hcc,length)
hcc_diag_2019_wide = dcast(hcc_diag_2019,MBI~hcc,length)
hcc_diag_2020_wide = dcast(hcc_diag_2020,MBI~hcc,length)

In [121]:
hcc_diag_2013_wide[MBI==100000015]
hcc_diag_2015_wide[MBI==100000015]
hcc_diag_2020_wide[MBI==100000015]

MBI,1,2,6,8,9,10,11,12,17,18,19,21,22,23,27,28,29,33,34,35,39,40,46,47,48,51,52,54,55,57,58,70,71,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,96,99,100,103,104,106,107,108,110,111,112,114,115,122,124,134,135,136,137,138,139,140,141,157,158,159,160,161,162,166,167,169,170,173,176,186,188,189
<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
100000015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


MBI,1,2,6,8,9,10,11,12,17,18,19,21,22,23,27,28,29,33,34,35,39,40,46,47,48,54,55,57,58,70,71,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,96,99,100,103,104,106,107,108,110,111,112,114,115,122,124,134,135,136,137,157,158,161,162,166,167,169,170,173,176,186,188,189
<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
100000015,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


MBI,1,2,6,8,9,10,11,12,17,18,19,21,22,23,27,28,29,33,34,35,39,40,46,47,48,54,55,57,58,70,71,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,96,99,100,103,104,106,107,108,110,111,112,114,115,122,124,134,135,136,137,157,158,161,162,166,167,169,170,173,176,186,188,189
<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
100000015,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [120]:
head(hcc_diag_2013_wide)
head(hcc_diag_2020_wide)

MBI,1,2,6,8,9,10,11,12,17,18,19,21,22,23,27,28,29,33,34,35,39,40,46,47,48,51,52,54,55,57,58,70,71,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,96,99,100,103,104,106,107,108,110,111,112,114,115,122,124,134,135,136,137,138,139,140,141,157,158,159,160,161,162,166,167,169,170,173,176,186,188,189
<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
100000015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100000019,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100000053,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100000099,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100000129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100000203,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


MBI,1,2,6,8,9,10,11,12,17,18,19,21,22,23,27,28,29,33,34,35,39,40,46,47,48,54,55,57,58,70,71,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,96,99,100,103,104,106,107,108,110,111,112,114,115,122,124,134,135,136,137,157,158,161,162,166,167,169,170,173,176,186,188,189
<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
100000015,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100000053,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100000203,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
100000241,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100000285,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100000327,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [119]:
write.csv(hcc_diag_2013_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2013_wide.csv")
write.csv(hcc_diag_2014_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2014_wide.csv")
write.csv(hcc_diag_2015_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2015_wide.csv")
write.csv(hcc_diag_2016_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2016_wide.csv")
write.csv(hcc_diag_2017_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2017_wide.csv")
write.csv(hcc_diag_2018_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2018_wide.csv")
write.csv(hcc_diag_2019_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2019_wide.csv")
write.csv(hcc_diag_2020_wide,"/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2020_wide.csv")

## Adding to the stable angina file

In [6]:
hcc_diag_2013_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2013_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2014_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2014_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2015_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2015_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2016_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2016_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2017_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2017_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2018_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2018_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2019_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2019_wide.csv",num_threads = numcores)%>%as.data.table()
hcc_diag_2020_wide=vroom("/work/postresearch/Shared/Projects/Farbod/hcc_data/wide_results/hcc_diag_2020_wide.csv",num_threads = numcores)%>%as.data.table()

New names:
• `` -> `...1`
Rows: 1229495 Columns: 89
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl (89): ...1, MBI, 1, 2, 6, 8, 9, 10, 11, 12, 17, 18, 19, 21, 22, 23, 27, 28, 29, 33, 34, 35, 39, 40, 46, 47, 48, 51, 52, 54, 55, 57, 58, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 96, 99, 100, 103, 104, 106, 107, 108, 110, 111, 112, 114, 115, 122, 124, 134, 135, 1...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 1196634 Columns: 81
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl (81): ...1, MBI, 1, 2, 6, 8, 9, 10, 11, 12, 17, 18, 19, 21, 22, 23, 27, 28, 29, 33, 3

In [7]:
yearly_calculations=vroom("yearly_calculations_with_tin.csv",num_threads = numcores)%>%as.data.table()

New names:
• `` -> `...1`
Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 14554 Columns: 597
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (227): ENTITLEMENT_BUY_IN_IND01_2013, ENTITLEMENT_BUY_IN_IND02_2013, ENTITLEMENT_BUY_IN_IND03_2013, ENTITLEMENT_BUY_IN_IND04_2013, ENTITLEMENT_BUY_IN_IND05_2013, ENTITLEMENT_BUY_IN_IND06_2013, ENTITLEMENT_BUY_IN_IND07_2013, ENTITLEMENT_BUY_IN_IND08_2013, ENTITLEMENT_BUY_IN_IND09_2013, ENTIT...
dbl  (339): ...1, X, DESY_SORT_KEY, tot_allowed_carrier, office_visit_count, office_visit_cost_carrier, cardiology_visit_count, distinct_clinicians, distinct_cardiologists, distinct_primary_care_physicians, distinct_surgeons, distinct_other_specialties, catheterization_count, catheterization_cos...
lgl   (25): stable_angina, unstable_angina, MI, cardiac_arrest, stroke, hypertension, copd, obesi

In [8]:
hcc_diag_2013_wide[,year:=2013]
hcc_diag_2014_wide[,year:=2014]
hcc_diag_2015_wide[,year:=2015]
hcc_diag_2016_wide[,year:=2016]
hcc_diag_2017_wide[,year:=2017]
hcc_diag_2018_wide[,year:=2018]
hcc_diag_2019_wide[,year:=2019]
hcc_diag_2020_wide[,year:=2020]

In [9]:
head(hcc_diag_2013_wide)

...1,MBI,1,2,6,8,9,10,11,12,17,18,19,21,22,23,27,28,29,33,34,35,39,40,46,47,48,51,52,54,55,57,58,70,71,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,96,99,100,103,104,106,107,108,110,111,112,114,115,122,124,134,135,136,137,138,139,140,141,157,158,159,160,161,162,166,167,169,170,173,176,186,188,189,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,100000015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2013
2,100000019,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2013
3,100000053,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2013
4,100000099,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2013
5,100000129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2013
6,100000203,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2013


In [10]:
hcc_diag_all_years=
rbind(hcc_diag_2013_wide,
      hcc_diag_2014_wide,
      hcc_diag_2015_wide,
      hcc_diag_2016_wide,
      hcc_diag_2017_wide,
      hcc_diag_2018_wide,
      hcc_diag_2019_wide,
      hcc_diag_2020_wide,fill=T)

In [12]:
colnames(hcc_diag_all_years)[-2]=paste("hcc",colnames(hcc_diag_all_years)[-2],sep="")

In [13]:
hcc_diag_all_years[is.na(hcc_diag_all_years)]<-0
hcc_diag_all_years=hcc_diag_all_years[,-1]

In [14]:
tail(hcc_diag_all_years)

MBI,hcc1,hcc2,hcc6,hcc8,hcc9,hcc10,hcc11,hcc12,hcc17,hcc18,hcc19,hcc21,hcc22,hcc23,hcc27,hcc28,hcc29,hcc33,hcc34,hcc35,hcc39,hcc40,hcc46,hcc47,hcc48,hcc51,hcc52,hcc54,hcc55,hcc57,hcc58,hcc70,hcc71,hcc72,hcc73,hcc74,hcc75,hcc76,hcc77,hcc78,hcc79,hcc80,hcc82,hcc83,hcc84,hcc85,hcc86,hcc87,hcc88,hcc96,hcc99,hcc100,hcc103,hcc104,hcc106,hcc107,hcc108,hcc110,hcc111,hcc112,hcc114,hcc115,hcc122,hcc124,hcc134,hcc135,hcc136,hcc137,hcc138,hcc139,hcc140,hcc141,hcc157,hcc158,hcc159,hcc160,hcc161,hcc162,hcc166,hcc167,hcc169,hcc170,hcc173,hcc176,hcc186,hcc188,hcc189,hccyear
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
499998435,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020
499999281,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020
499999461,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020
499999521,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020
499999841,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020
499999981,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020


In [15]:
yearly_calculations[,year_before_diagnosis:=year_first_diagnosed-1]
head(yearly_calculations)

...1,X,DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,MI_date,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_

In [16]:
yearly_calculations_with_diag=left_join(yearly_calculations,hcc_diag_all_years,by=c("DESY_SORT_KEY"="MBI","year_before_diagnosis"="hccyear" ))%>%as.data.table()

In [20]:
yearly_calculations_with_diag=yearly_calculations_with_diag[,-c("...1","X")]

In [21]:
head(yearly_calculations_with_diag)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,MI_date,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HM

In [22]:
write.csv(yearly_calculations_with_diag,"yearly_calculations_with_diag.csv")

# For person file

In [97]:
mbsf_2013=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2013.csv" , num_threads = numcores) %>% as.data.table
mbsf_2014=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2014.csv" , num_threads = numcores) %>% as.data.table
mbsf_2015=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2015.csv" , num_threads = numcores) %>% as.data.table
mbsf_2016=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2016.csv" , num_threads = numcores) %>% as.data.table
mbsf_2017=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2017.csv" , num_threads = numcores) %>% as.data.table
mbsf_2018=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2018.csv" , num_threads = numcores) %>% as.data.table
mbsf_2019=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2019.csv" , num_threads = numcores) %>% as.data.table
mbsf_2020=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2020.csv" , num_threads = numcores) %>% as.data.table


New names:
• `06071` -> `06071...6`
• `06071` -> `06071...7`
• `06071` -> `06071...8`
• `06071` -> `06071...9`
• `06071` -> `06071...10`
• `06071` -> `06071...11`
• `06071` -> `06071...12`
• `06071` -> `06071...13`
• `06071` -> `06071...14`
• `06071` -> `06071...15`
• `06071` -> `06071...16`
• `06071` -> `06071...17`
• `0` -> `0...21`
• `0` -> `0...22`
• `0` -> `0...23`
• `10` -> `10...24`
• `10` -> `10...25`
• `10` -> `10...26`
• `10` -> `10...27`
• `10` -> `10...28`
• `10` -> `10...29`
• `10` -> `10...30`
• `10` -> `10...31`
• `10` -> `10...32`
• `10` -> `10...33`
• `10` -> `10...34`
• `10` -> `10...35`
• `0` -> `0...36`
• `0` -> `0...37`
• `3` -> `3...38`
• `3` -> `3...39`
• `3` -> `3...40`
• `3` -> `3...41`
• `3` -> `3...42`
• `3` -> `3...43`
• `3` -> `3...44`
• `3` -> `3...45`
• `3` -> `3...46`
• `3` -> `3...47`
• `3` -> `3...48`
• `3` -> `3...49`
• `C` -> `C...50`
• `C` -> `C...51`
• `C` -> `C...52`
• `C` -> `C...53`
• `C` -> `C...54`
• `C` -> `C...55`
• `C` -> `C...56`
• `C` -> 

In [98]:
mbsf_colnames_2013_2015=c("DESY_SORT_KEY","STATE_CODE","COUNTY_CODE","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MEDICARE_STATUS_CD","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","REFERENCE_YEAR")

mbsf_colnames_2016_2020=c("DESY_SORT_KEY","REFERENCE_YEAR","SAMPLE_GROUP","STATE_CODE","COUNTY_CODE","STATE_CNTY_FIPS_CD_01","STATE_CNTY_FIPS_CD_02","STATE_CNTY_FIPS_CD_03","STATE_CNTY_FIPS_CD_04","STATE_CNTY_FIPS_CD_05","STATE_CNTY_FIPS_CD_06","STATE_CNTY_FIPS_CD_07","STATE_CNTY_FIPS_CD_08","STATE_CNTY_FIPS_CD_09","STATE_CNTY_FIPS_CD_10","STATE_CNTY_FIPS_CD_11","STATE_CNTY_FIPS_CD_12","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MDCR_STATUS_CODE_01","MDCR_STATUS_CODE_02","MDCR_STATUS_CODE_03","MDCR_STATUS_CODE_04","MDCR_STATUS_CODE_05","MDCR_STATUS_CODE_06","MDCR_STATUS_CODE_07","MDCR_STATUS_CODE_08","MDCR_STATUS_CODE_09","MDCR_STATUS_CODE_10","MDCR_STATUS_CODE_11","MDCR_STATUS_CODE_12","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","DUAL_STUS_CD_01","DUAL_STUS_CD_02","DUAL_STUS_CD_03","DUAL_STUS_CD_04","DUAL_STUS_CD_05","DUAL_STUS_CD_06","DUAL_STUS_CD_07","DUAL_STUS_CD_08","DUAL_STUS_CD_09","DUAL_STUS_CD_10","DUAL_STUS_CD_11","DUAL_STUS_CD_12")


colnames(mbsf_2013)=mbsf_colnames_2013_2015
colnames(mbsf_2014)=mbsf_colnames_2013_2015
colnames(mbsf_2015)=mbsf_colnames_2013_2015
colnames(mbsf_2016)=mbsf_colnames_2016_2020
colnames(mbsf_2017)=mbsf_colnames_2016_2020
colnames(mbsf_2018)=mbsf_colnames_2016_2020
colnames(mbsf_2019)=mbsf_colnames_2016_2020
colnames(mbsf_2020)=mbsf_colnames_2016_2020

In [104]:
mbsf_colnames_2013_2015

[1] "DESY_SORT_KEY"               "STATE_CODE"                  "COUNTY_CODE"                 "SEX_CODE"                    "RACE_CODE"                   "AGE"                         "ORIG_REASON_FOR_ENTITLEMENT" "CURR_REASON_FOR_ENTITLEMENT" "ESRD_INDICATOR"             
[10] "MEDICARE_STATUS_CD"          "PART_A_TERMINATION_CODE"     "PART_B_TERMINATION_CODE"     "ENTITLEMENT_BUY_IN_IND01"    "ENTITLEMENT_BUY_IN_IND02"    "ENTITLEMENT_BUY_IN_IND03"    "ENTITLEMENT_BUY_IN_IND04"    "ENTITLEMENT_BUY_IN_IND05"    "ENTITLEMENT_BUY_IN_IND06"   
[19] "ENTITLEMENT_BUY_IN_IND07"    "ENTITLEMENT_BUY_IN_IND08"    "ENTITLEMENT_BUY_IN_IND09"    "ENTITLEMENT_BUY_IN_IND10"    "ENTITLEMENT_BUY_IN_IND11"    "ENTITLEMENT_BUY_IN_IND12"    "HMO_INDICATOR01"             "HMO_INDICATOR02"             "HMO_INDICATOR03"            
[28] "HMO_INDICATOR04"             "HMO_INDICATOR05"             "HMO_INDICATOR06"             "HMO_INDICATOR07"             "HMO_INDICATOR08"             "HMO_INDICATOR09"             "HMO_INDICATOR10"             "HMO_INDICATOR11"             "HMO_INDICATOR12"            
[37] "HI_COVERAGE"                 "SMI_COVERAGE"                "HMO_COVERAGE"                "STATE_BUY_IN_COVERAGE"       "VALID_DATE_OF_DEATH_SWITCH"  "DATE_OF_DEATH"               "REFERENCE_YEAR"

In [101]:
head(mbsf_2013)
head(mbsf_2020)

nrow(mbsf_2013)
nrow(mbsf_2020)


DESY_SORT_KEY,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ESRD_INDICATOR,MEDICARE_STATUS_CD,PART_A_TERMINATION_CODE,PART_B_TERMINATION_CODE,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,HI_COVERAGE,SMI_COVERAGE,HMO_COVERAGE,STATE_BUY_IN_COVERAGE,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,REFERENCE_YEAR
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
000000000,45,910,2,1,75,0,0,0,10,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0,11,NA,NA,13
000000000,22,170,2,1,71,0,0,0,10,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0,0,NA,NA,13
000000000,33,420,2,1,93,0,0,0,10,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0,0,NA,NA,13
000000000,49,801,2,1,71,0,0,0,10,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,NA,NA,13
000000000,33,400,2,1,75,0,0,0,10,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0,0,NA,NA,13
000000000,10,510,1,1,70,0,0,0,10,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0,0,NA,NA,13


DESY_SORT_KEY,REFERENCE_YEAR,SAMPLE_GROUP,STATE_CODE,COUNTY_CODE,STATE_CNTY_FIPS_CD_01,STATE_CNTY_FIPS_CD_02,STATE_CNTY_FIPS_CD_03,STATE_CNTY_FIPS_CD_04,STATE_CNTY_FIPS_CD_05,STATE_CNTY_FIPS_CD_06,STATE_CNTY_FIPS_CD_07,STATE_CNTY_FIPS_CD_08,STATE_CNTY_FIPS_CD_09,STATE_CNTY_FIPS_CD_10,STATE_CNTY_FIPS_CD_11,STATE_CNTY_FIPS_CD_12,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ESRD_INDICATOR,MDCR_STATUS_CODE_01,MDCR_STATUS_CODE_02,MDCR_STATUS_CODE_03,MDCR_STATUS_CODE_04,MDCR_STATUS_CODE_05,MDCR_STATUS_CODE_06,MDCR_STATUS_CODE_07,MDCR_STATUS_CODE_08,MDCR_STATUS_CODE_09,MDCR_STATUS_CODE_10,MDCR_STATUS_CODE_11,MDCR_STATUS_CODE_12,PART_A_TERMINATION_CODE,PART_B_TERMINATION_CODE,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,HI_COVERAGE,SMI_COVERAGE,HMO_COVERAGE,STATE_BUY_IN_COVERAGE,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,DUAL_STUS_CD_01,DUAL_STUS_CD_02,DUAL_STUS_CD_03,DUAL_STUS_CD_04,DUAL_STUS_CD_05,DUAL_STUS_CD_06,DUAL_STUS_CD_07,DUAL_STUS_CD_08,DUAL_STUS_CD_09,DUAL_STUS_CD_10,DUAL_STUS_CD_11,DUAL_STUS_CD_12
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
100000015,2020,04,22,160,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,1,1,83,0,0,0,10,10,10,10,10,10,10,10,10,10,10,10,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
100000053,2020,01,33,331,36047,36047,36047,36047,36047,36047,36047,36047,36047,36047,36047,36047,1,2,84,0,0,0,10,10,10,10,10,10,10,10,10,10,10,10,0,0,C,C,C,C,C,C,C,C,C,C,C,C,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0,12,NA,NA,02,02,02,02,02,02,02,02,02,02,02,02
100000099,2020,04,10,350,12071,12071,12071,12071,12071,12071,12071,12071,12071,12071,12071,12071,1,1,89,0,0,0,10,10,10,10,10,10,10,10,10,10,10,10,0,0,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,12,12,12,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
100000113,2020,01,55,330,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,1,3,86,0,0,0,10,10,10,10,10,10,10,10,10,10,10,10,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
100000129,2020,04,07,040,09003,09003,09003,09003,09003,09003,09003,09003,09003,09003,09007,09007,1,1,59,1,1,0,20,20,20,20,20,20,20,20,20,20,20,20,0,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,12,12,12,12,NA,NA,01,01,01,01,01,01,01,01,02,02,02,02
100000133,2020,01,55,330,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,2,3,84,0,0,0,10,10,10,10,10,10,10,10,10,10,10,10,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


[1] 2758309

[1] 3296239

In [105]:
create_person_file_2013_2015=function(mbsf_data){
    
    result=mbsf_data[,
          .(
              MBI=DESY_SORT_KEY,
              SEX=SEX_CODE,
              DOB=as.IDate(paste(
                  as.character(REFERENCE_YEAR-AGE),"-01","-01",sep="")
                       ,"%Y-%m-%d"),
              OREC=ORIG_REASON_FOR_ENTITLEMENT,
              LTIMCAID=0,
              NEMCAID=0
          ),keyby=DESY_SORT_KEY
         ]
    
    result=result[order(MBI)]
    return(result)
}

create_person_file_2016_2020=function(mbsf_data){
    
    result=mbsf_data[,
          .(
              MBI=DESY_SORT_KEY,
              SEX=SEX_CODE,
              DOB=as.IDate(paste(
                  as.character(REFERENCE_YEAR-AGE),"-01","-01",sep="")
                       ,"%Y-%m-%d"),
              OREC=ORIG_REASON_FOR_ENTITLEMENT,
              LTIMCAID=(sum(
                  DUAL_STUS_CD_01 %in% c('02','04','08'),
                  DUAL_STUS_CD_02 %in% c('02','04','08'),
                  DUAL_STUS_CD_03 %in% c('02','04','08'),
                  DUAL_STUS_CD_04 %in% c('02','04','08'),
                  DUAL_STUS_CD_05 %in% c('02','04','08'),
                  DUAL_STUS_CD_06 %in% c('02','04','08'),
                  DUAL_STUS_CD_07 %in% c('02','04','08'),
                  DUAL_STUS_CD_08 %in% c('02','04','08'),
                  DUAL_STUS_CD_09 %in% c('02','04','08'),
                  DUAL_STUS_CD_10 %in% c('02','04','08'),
                  DUAL_STUS_CD_11 %in% c('02','04','08'),
                  DUAL_STUS_CD_12 %in% c('02','04','08'),na.rm=T
              )>0)*1,
              NEMCAID=0
          ),keyby=DESY_SORT_KEY
         ]
    
    result=result[order(MBI)]
    return(result)
}

In [106]:
person_file_2013=create_person_file_2013_2015(mbsf_2013)
person_file_2014=create_person_file_2013_2015(mbsf_2014)
person_file_2015=create_person_file_2013_2015(mbsf_2015)
person_file_2016=create_person_file_2016_2020(mbsf_2016)
person_file_2017=create_person_file_2016_2020(mbsf_2017)
person_file_2018=create_person_file_2016_2020(mbsf_2018)
person_file_2019=create_person_file_2016_2020(mbsf_2019)
person_file_2020=create_person_file_2016_2020(mbsf_2020)

In [134]:
write.csv(person_file_2013,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2013.csv")
write.csv(person_file_2014,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2014.csv")
write.csv(person_file_2015,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2015.csv")
write.csv(person_file_2016,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2016.csv")
write.csv(person_file_2017,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2017.csv")
write.csv(person_file_2018,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2018.csv")
write.csv(person_file_2019,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2019.csv")
write.csv(person_file_2020,"/work/postresearch/Shared/Projects/Farbod/hcc_data/person_file_2020.csv")